# Домашнее задание № 7

### Задание 1 Реализовать алгоритм Леска и проверить его на реальном датасете (8 баллов)

Ворднет можно использовать для дизамбигуации. Самый простой алгоритм дизамбигуации - алгоритм Леска. В нём нужное значение слова находится через пересечение слов контекста, в котором употреблено это слово, с определениями значений слова из ворднета. Значение с максимальным пересечением - нужное.

Реализуйте его

In [ ]:
from string import punctuation
punct = punctuation+'«»—…“”*№–'
import nltk
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

In [127]:
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

Для начала создадим вспомогательные функции.  
Функция приведения слова к изначальной форме:

In [131]:
def word_to_initial_form(word):
    return lemmatizer.lemmatize(word)

Функция, которая переводит предложение в список изначальных форм слов:

In [132]:
def sent_to_list_of_initial_forms(sentence):
    words = [token for token in list(word_tokenize(sentence)) if token not in punct]
    words = [lemmatizer.lemmatize(word) for word in words if word]
    return words

Функция, которая возвращает список 5 слов до и 5 слов после целевого слова, если столько есть в предложении:

In [166]:
def get_words_in_context(sentence, target_word):
    
    words = [token for token in list(word_tokenize(sentence)) if token not in punct]
    words = [lemmatizer.lemmatize(word) for word in words if word]

    target_word_initial_form = lemmatizer.lemmatize(target_word)
    
    for i in range(len(words)):
        if words[i] == target_word_initial_form:
            break
    context = []
    j = 1
    while j<6 and (i-j)>=0:
        context.append(words[i-j])
        j+=1
    j = 1
    while j<6 and j+i<len(words):
        context.append(words[i+j])
        j+=1
    return context

Функция lesk, которая принимает на вход целевое слово и содержащее его предложение и возвращает индекс наиболее подходящего определения из Wordnet:

In [182]:
def lesk(word, sentence):

    list_of_coefficients = []
    synsets_to_compare = wn.synsets(word_to_initial_form(word))
    for i in range(len(synsets_to_compare)):
        counter = 0
        for context_word in get_words_in_context(sentence, word):
            if context_word in sent_to_list_of_initial_forms(wn.synsets(word_to_initial_form(word))[i].definition()):
                counter += 1
        list_of_coefficients.append(counter)    
        
    best_synset_index = list_of_coefficients.index(max(list_of_coefficients))
        
    return best_synset_index

In [186]:
# на вход подается элемент результата работы уже написанной вами функции get_words_in_context
lesk('day', 'some point or period in time') # для примера контекст совпадает с одним из определений
# а на выходе индекс подходящего синсета

1

In [187]:
wn.synsets('day')

[Synset('day.n.01'),
 Synset('day.n.02'),
 Synset('day.n.03'),
 Synset('day.n.04'),
 Synset('day.n.05'),
 Synset('day.n.06'),
 Synset('day.n.07'),
 Synset('sidereal_day.n.01'),
 Synset('day.n.09'),
 Synset('day.n.10')]

In [188]:
# с помощью этого индекса достаем нужный синсет
wn.synsets('day')[1].definition()

'some point or period in time'

In [189]:
wn.synsets('day')[1]

Synset('day.n.02')

**Проверьте насколько хорошо работает такой метод на датасете из семинара**

In [140]:
corpus_wsd = []
corpus = open('corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

Корпус состоит из предложений, где у каждого слова три поля - значение, лемма и само слово. Значение пустое, когда слово однозначное, а у многозначных слов стоит тэг вида **'long%3:00:02::'** Это тэг wordnet'ного формата

**Вам нужно для каждого многозначного слова (т.е. у него есть тэг в первом поле) с помощью алгоритма Леска предсказать нужный синсет и сравнить с правильным. Посчитайте процент правильных предсказаний (accuracy).**

Если считается слишком долго, возьмите поменьше предложений (например, только тысячу)

In [141]:
len(corpus_wsd)

49453

In [142]:
corpus_wsd[0]

[['', 'how', 'How'],
 ['long%3:00:02::', 'long', 'long'],
 ['', 'have', 'has'],
 ['', 'it', 'it'],
 ['be%2:42:03::', 'be', 'been'],
 ['', 'since', 'since'],
 ['', 'you', 'you'],
 ['review%2:31:00::', 'review', 'reviewed'],
 ['', 'the', 'the'],
 ['objective%1:09:00::', 'objective', 'objectives'],
 ['', 'of', 'of'],
 ['', 'you', 'your'],
 ['benefit%1:21:00::', 'benefit', 'benefit'],
 ['', 'and', 'and'],
 ['service%1:04:07::', 'service', 'service'],
 ['program%1:09:01::', 'program', 'program'],
 ['', '?', '?']]

Определим функцию, которая достаёт предложения из корпуса:

In [153]:
def sentence_out_of_corpus(corpus_item):
    sentence = ''
    for i in corpus_item:
        sentence = sentence + i[2] + " "
    return sentence

Для первой тысячи предложений добавим в список результаты сравнения выбранного алгоритмом Леска определения с заявленным:

In [ ]:
accuracy_check = []

for num_sent in range(1000):
    for word_info in corpus_wsd[num_sent]:
        if word_info[0] != '':
            accuracy_check.append(wn.lemma_from_key(word_info[0]).synset() == wn.synsets(word_info[1])\
                                  [lesk(word_info[1], sentence_out_of_corpus(corpus_wsd[num_sent]))])
accuracy_check

In [184]:
len(accuracy_check)

9306

In [185]:
accuracy = sum(accuracy_check)/len(accuracy_check)
accuracy

0.41629056522673547

### Задание 2* (2 балла)

В семинаре для WSI на данных Диалога использовался только Fastext. Попробуйте заменить его на адаграм (обучите свою модель или используйте предобученную out.pkl или https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib), а также поэкспериментируйте с разными алгоритмами кластеризации и их параметрами (можно использовать только те алгоритмы, которые обучаются достаточно быстро)

Для каждого эксперимента рассчитайте ARI